## 1.什么是随机森林

### 1.1 随机森林

Random Forest (随机森林) 是一种基于树模型的 Bagging 的优化版本，每一棵树都不完美，但 n 棵不完美的树组合在一起，覆盖彼此缺点。因此就有了随机森林，解决决策树泛化能力弱的特点。(可以理解成三个臭皮匠顶过诸葛亮)

而同一批数据，用同样的算法只能产生一棵树，这时Bagging策略可以帮助我们产生不同的数据集。

### 1.2 Bagging思想

**Bagging** 策略来源于bootstrap aggregation。思想就是从总体样本当中随机取一部分样本进行训练，通过多次这样的结果，进行投票获取平均值作为结果输出。

**举个例子**：

假设有 1000 个样本，如果按照以前的思维，是直接把这 1000 个样本拿来训练，但现在不一样，先抽取 800 个样本来进行训练，重复以上操作（再抽取 800 样本训练），提高模型输出的平均值。

1. 从样本集（$N$ 个数据点）中，随机采样选出 $N_b$ 个样本。
2. 在所有样本上，对这 $N_b$ 个样本建立分类器（ID3\C4.5\CART\SVM\LOGISTIC）

重复以上两步 m 次，获得 m 个分类器，最后根据这 m 个分类器的投票结果，决定数据属于哪一类。

**每棵树的按照如下规则生成：**

1. 如果训练集大小为 $N$，对于每棵树而言，**随机**且有放回地从训练集中的抽取 $N_b$ 个训练样本，作为该树的训练集；
2. 如果每个样本的特征维度为 M，指定一个常数 m<<M，**随机**地从 M 个特征中选取 m 个特征子集，每次树进行分裂时，从这 m 个特征中选择最优的；
3. 每棵树都尽最大程度的生长，并且没有剪枝过程。

一开始我们提到的随机森林中的“随机”就是指的这里的两个随机性。两个随机性的引入对随机森林的分类性能至关重要。由于它们的引入，使得随机森林不容易陷入过拟合，并且具有很好得抗噪能力（比如：对缺省值不敏感）。

总的来说就是随机选择样本数，随机选取特征，随机选择分类器，建立多颗这样的决策树，然后通过这几课决策树来投票，决定数据属于哪一类(**投票机制有一票否决制、少数服从多数、加权多数**)

## 2. 随机森林分类效果的影响因素

- 森林中任意两棵树的相关性：相关性越大，错误率越大；
- 森林中每棵树的分类能力：每棵树的分类能力越强，整个森林的错误率越低。

减小特征选择个数 m，树的相关性和分类能力也会相应的降低；增大 m，两者也会随之增大。所以关键问题是如何选择最优的 m（或者是范围），这也是随机森林唯一的一个参数。

## 3. 随机森林有什么优缺点

**优点：**

- 在当前的很多数据集上，相对其他算法有着很大的优势，表现良好。
- 它能够处理很高维度（feature 很多）的数据，并且不用做特征选择(因为特征子集是随机选择的)。
- 在训练完后，它能够给出哪些 feature 比较重要。
- 训练速度快，容易做成并行化方法(训练时树与树之间是相互独立的)。
- 在训练过程中，能够检测到 feature 间的互相影响。
- 对于不平衡的数据集来说，它可以平衡误差。
- 如果有很大一部分的特征遗失，仍可以维持准确度。

**缺点：**

- 随机森林已经被证明在某些**噪音较大**的分类或回归问题上会过拟合。
- 对于有不同取值的属性的数据，取值划分较多的属性会对随机森林产生更大的影响，所以随机森林在这种数据上产出的属性权值是不可信的。

## 4. 随机森林如何处理缺失值？

根据随机森林创建和训练的特点，随机森林对缺失值的处理还是比较特殊的。

- 首先，给缺失值预设一些估计值，比如数值型特征，选择其余数据的中位数或众数作为当前的估计值
- 然后，根据估计的数值，建立随机森林，把所有的数据放进随机森林里面跑一遍。记录每一组数据在决策树中一步一步分类的路径.
- 判断哪组数据和缺失数据路径最相似，引入一个相似度矩阵，来记录数据之间的相似度，比如有N组数据，相似度矩阵大小就是 $N*N$
- 如果缺失值是类别变量，通过权重投票得到新估计值，如果是数值型变量，通过加权平均得到新的估计值，如此迭代，直到得到稳定的估计值。

其实，该缺失值填补过程类似于推荐系统中采用协同过滤进行评分预测，先计算缺失特征与其他特征的相似度，再加权得到缺失值的估计，而随机森林中计算相似度的方法（数据在决策树中一步一步分类的路径）乃其独特之处。

## 5. 什么是OOB？随机森林中OOB是如何计算的，它有什么优缺点？

**OOB**：

上面我们提到，构建随机森林的关键问题就是如何选择最优的 m，要解决这个问题主要依据计算袋外错误率 oob error（out-of-bag error）。

bagging 方法中 Bootstrap 每次约有 1/3 的样本不会出现在 Bootstrap 所采集的样本集合中，当然也就没有参加决策树的建立，把这 1/3 的数据称为**袋外数据 oob（out of bag）**,它可以用于取代测试集误差估计方法。

**袋外数据 (oob) 误差的计算方法如下：**

- 对于已经生成的随机森林,用袋外数据测试其性能,假设袋外数据总数为 O,用这 O 个袋外数据作为输入,带进之前已经生成的随机森林分类器,分类器会给出 O 个数据相应的分类
- 因为这 O 条数据的类型是已知的,则用正确的分类与随机森林分类器的结果进行比较,统计随机森林分类器分类错误的数目,设为 X,则袋外数据误差大小 = X/O

**优缺点**：

这已经经过证明是无偏估计的,所以在随机森林算法中不需要再进行交叉验证或者单独的测试集来获取测试集误差的无偏估计。

## 6. 随机森林的过拟合问题

你已经建了一个有 10000 棵树的随机森林模型。在得到 0.00 的训练误差后，你非常高兴。但是，验证错误是 34.23。到底是怎么回事？你还没有训练好你的模型吗？

答：该模型过度拟合，因此，为了避免这些情况，我们要用交叉验证来调整树的数量。

## 7. 代码实现